## 7.3 プーリング層

![CNNによるネットワークの例](fig/CNNによるネットワークの例.png)

◆Maxプーリングの処理手順 <br>
![Maxプーリングの処理手順](fig/Maxプーリングの処理手順.png)<br>
上記例では2×2の領域に対して最大となる要素を取り出している。<br>
*Maxプーリングの他に、Averageプーリング(対象領域の平均)等あり。<br>
*画像認識の分野では主にMaxプーリングが使用される。<br>
*その為、本書で「プーリング層」という場合は、Maxプーリングを指すものとする。<br>
*プーリングとは
https://toukeigaku-jouhou.info/2018/04/22/pooling/

### 7.3.1 プーリング層の特徴

(1)学習するパラメータがない<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
対象領域から最大値(もしくは平均値)を取るだけの処理<br><br>
(2)チャンネル数は変化しない<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
プーリングの演算によって入力データと出力データのチャンネル数は変化しない<br>
![プーリングではチャンネル数は変わらない](fig/プーリングではチャンネル数は変わらない.png)<br><br>
(3)微小な位置変化に対してロバスト(頑健)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
入力データの小さなズレに対して、プーリングは同じような結果を返す。<br>
![微小な位置変化に対してロバスト](fig/微小な位置変化に対してロバスト.png)<br>
上図のような3×3のプーリングの場合、入力データが横方向に1要素分だけずれた場合でも、出力は同じような結果になる。<br>

## 7.4 Convolution/Poolingレイヤの実装

畳み込み層とプーリング層の実装を行なっていく。<br>
ここで実装するクラスにも、forwardとbackwardというメソッドを持たせ、モジュールとして利用できるようにする。<br>

### 7.4.1 4次元配列

<h4>復習

<!-- ◆出力が1枚の特徴量マップ(=チャンネル数が1の特徴マップ)<br>
![出力が1枚の特徴マップ](fig/出力が1枚の特徴マップ.png)<br><br>
◆出力が複数(=チャンネル数が複数)<br>
![出力が複数](fig/出力が複数.png)<br> -->
<!-- ◆バイアス項も追加<br><br> -->
![畳み込み演算の処理フロー](fig/畳み込み演算の処理フロー.png)<br>

データの形状が(10, 1, 28, 28)の場合、高さ28・横幅28で１チャンネルのデータが10個ある場合に対応

In [13]:
import numpy as np
x=np.random.rand(10,1,28,28)
x.shape

(10, 1, 28, 28)

In [14]:
print(f'x[0].shape : {x[0].shape}\n')
print(f'x[1].shape : {x[1].shape}\n')

x[0].shape : (1, 28, 28)

x[1].shape : (1, 28, 28)



### 7.4.2 im2colによる展開

◆ゼロから作るDeep Learning 3より<br><br>
![ゼロから作るDeepLearning3](fig/ゼロから作るDeepLearning3.png)<br>
◆入力データとカーネルの行列の積による計算<br><br>
![入力データとカーネルの行列の積による計算](fig/入力データとカーネルの行列の積による計算.png)<br>

以下参考URL<br>
https://qiita.com/omiita/items/01855ff13cc6d3720ea4<br>
https://qiita.com/daizutabi/items/856042fb1ea9da150741<br>
https://qiita.com/jun40vn/items/be48afbdd9da19f1e43e<br>

◆4次元配列<br><br>
![4次元配列](fig/4次元配列.png)<br><br>
上記例は高さ2,幅3の画像が、チャネル数2(横方向),バッチ数3(縦方向)で並んでると解釈できる

◆im2colによる展開<br>
 フィルタサイズがFH=2, FW=2の場合にim2col関数でXを2次元配列に変換した結果<br>
![im2colによる展開](fig/im2colによる展開.png)<br>

◆im2colによる展開(色付き)<br>
 フィルタサイズがFH=2, FW2の場合にim2col関数でXを2次元配列に変換した結果<br>
![im2colによる展開色付き](fig/im2colによる展開色付き.png)<br>

◆im2colによる展開(図付き)<br>
![四次元から２次元へ](fig/四次元から２次元へ.png)<br>

In [10]:
import numpy as np
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """
    Parameters
    ----------
    input_data : (データ数, チャンネル, 高さ, 幅)の4次元配列からなる入力データ
    filter_h : フィルターの高さ
    filter_w : フィルターの幅
    stride : ストライド
    pad : パディング

    Returns
    -------
    col : 2次元配列
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [11]:
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [12]:
x2 = np.random.rand(10, 3, 7, 7) # 10 個のデータ
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


### 7.4.3 Convolutionレイヤの実装

In [3]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        """Convolutionレイヤー

        Args:
            W (numpy.ndarray): フィルター（重み）、形状は(FN, C, FH, FW)。
            b (numpy.ndarray): バイアス、形状は(FN)。
            stride (int, optional): ストライド、デフォルトは1。
            pad (int, optional): パディング、デフォルトは0。
        """     
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
   
        # 中間データ（backward時に使用）
        self.x = None   
        self.col = None
        self.col_W = None
        
        # 重み・バイアスパラメータの勾配
        self.dW = None
        self.db = None

    def forward(self, x):
        """順伝播

        Args:
            x (numpy.ndarray): 入力。形状は(N, C, H, W)。

        Returns:
            numpy.ndarray: 出力。形状は(N, FN, OH, OW)。
        """       
        FN, C, FH, FW = self.W.shape  # FN:フィルター数、C:チャンネル数、FH:フィルターの高さ、FW:幅
        N, C, H, W = x.shape  # N:バッチサイズ、x_C:チャンネル数、H：入力データの高さ、W:幅
        
        # ①出力サイズの計算
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        # ②im2colで画像データを行列データに変換
        # (N, C, H, W) → (N * OH * OW, C * FH * FW)
        col = im2col(x, FH, FW, self.stride, self.pad)
        
        # ③フィルターをreshapeして2次元配列に展開
        # (FN, C, FH, FW) → (C * FH * FW, FN)
        col_W = self.W.reshape(FN, -1).T

        # ④行列演算で出力を計算
        # 出力を算出（col_x, col_W, bに対する計算は、Affineレイヤーと全く同じ）
        # (N * OH * OW, C * FH * FW)・(C * FH * FW, FN) → (N * OH * OW, FN)
        out = np.dot(col, col_W) + self.b2
        
        # ⑤出力の形を整える
        # (N * OH * OW, FN) → (N, OH, OW, FN) → (N, FN, OH, OW)
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        # 逆伝播のために保存
        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

①出力サイズの計算式<br>
![出力サイズの計算式](fig/出力サイズの計算式.png)<br>

③フィルターをreshapeして2次元配列に展開<br>
![フィルターreshape](fig/フィルターreshape.png)<br>

![X*W](fig/XW.png)<br>

④行列演算で出力を計算

<h4> Ŷ =X̂ ⋅Ŵ +B

<table>
<thead><tr>
<th style="text-align:left">2次元化した配列</th>
<th style="text-align:left">行数</th>
<th style="text-align:left">列数</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left">X̂</td>
<td style="text-align:left">N×OH×OW</td>
<td style="text-align:left">C×FH×FW</td>
</tr>
<tr>
<td style="text-align:left">Ŵ</td>
<td style="text-align:left">C×FH×FW</td>
<td style="text-align:left">FN</td>
</tr>
<tr>
<td style="text-align:left">Ŷ</td>
<td style="text-align:left">N×OH×OW</td>
<td style="text-align:left">FN</td>
</tr>
</tbody>
</table>

### 7.4.4 Poolingレイヤの実装

In [13]:

class Pooling:
    def __init__(self, pool_h, pool_w, stride=2, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx
